## Open Street Map Data Wrangling Project

### Location: [Edinburgh Scotland](http://www.openstreetmap.org/export#map=14/55.9427/-3.1858)

By complete chance my wife and two good friends ended up here on the same days in the same hotel.  The best I can do is this map.

The data file for the whole city of Edinburgh was rather large so I took a manually selected subset of the city and exported it using the [Overpass API](http://overpass-api.de/api/map?bbox=-3.2261,55.9312,-3.1595,55.9562).


### Importing the schema to MySQL

The provided .sql had to be modified for use with MySQL because the column names **key** and **value** are reserved words in MySQL.  They simply need to be escaped by surrounding them in back-ticks (e.g. **\`key\`** and **\`value\`**) in the .sql file.

From the command line:
```bash
mysql -u username -p database_name < schema_file.sql
```
[StackOverflow](https://stackoverflow.com/questions/17666249/how-to-import-an-sql-file-using-the-command-line-in-mysql)

### Importing the csv files to MySQL

From command line (repeat for each of the csv files):
```sql
mysqlimport --ignore-lines=1 --fields-terminated-by=, --verbose --local -u root -p open_street_map file.csv
```
[Import csv to MySQL](https://www.garron.me/en/bits/mysql-import-csv.html)

### Data Issues and Questions
* Skipped nodes records on import to MySQL
* Predominance of node_tags <mark>k="source"</mark> and slight variations of the <mark>v="values"</mark>
  


### Skipped node records on import to MySQL
 * As a result of using the <mark>--verbose</mark> flag during import I was able to see that ***165,602*** out of ***195,524*** nodes records were skipped (i.e. not imported into the table).  After a little investigation of the raw nodes.csv file I noticed that the <mark>id</mark> values were very large.  I looked in the <mark>open_map_project_schema.sql</mark> file and noticed that the <mark>id</mark> fields were set to <mark>INT</mark>.  After looking at the MySQL documentation I was able to determine that the <mark>INT</mark> type can only store values up to ***4,294,967,295*** assuming the field is <mark>UNSIGNED</mark>.  The schema file only specified <mark>INTEGER</mark> so the limit was actually ***2,147,483,647*** and in either case I had id values that were larger.  
  I changed all of the columns that were related to ids to the <mark>BIGINT</mark> type.  After that change all nodes loaded successfully. 

### Node tags where k="source"
While exploring the <mark>nodes_tags</mark> table I noticed that **34,017** out of **39,025** or **87%** of the <mark>key</mark> values were <mark>source</mark>.

```sql
SELECT `key`, COUNT(*) as cnt
FROM nodes_tags
GROUP BY `key`
ORDER BY cnt DESC;
```
Looking at the distribution of values within the <mark>keys</map> we see that the vast majority of values, **32,321** are **survey** (**18,290**) and **Bing** (**14,031**). In the remainder of the list there are some minor issues.  For example, there are records with values of <mark>naptan_import;survey</mark>, <mark>naptan_import; survey</mark>, <mark>naptan_import/survey</mark>, and <mark>naptan_import;survet</mark>. 

```sql
SELECT `value`, COUNT(*) as cnt
FROM nodes_tags
WHERE `key`='source'
GROUP BY cnt DESC;
```